In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '7'

In [2]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import json

from torch.utils.data import Dataset,DataLoader
import torch
from torch import nn
from torch.nn.modules.loss import _WeightedLoss
import timm
import time
from tqdm import tqdm
import cv2
import numpy as np
#from torch.cuda.amp import autocast, GradScaler

import matplotlib.pyplot as plt
import random

%matplotlib inline

In [3]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)
from albumentations.pytorch import ToTensorV2

In [4]:
test_fold = 0
#img_size = 512
#bs = 16
epochs_to_predict = [7,8,9]

In [5]:
CFG = {
    'fold_num': 0,
    'seed': 2021,
    'model_arch': 'tf_efficientnet_b4_ns',
    'img_size': 512,
    'epochs': 10,
    'train_bs': 16,
    'valid_bs': 32,
    'T_0': 10,
    'lr': 1e-4,
    'min_lr': 1e-6,
    'weight_decay':1e-6,
    'num_workers': 8,
    'accum_iter': 2, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:0'
}

In [6]:
TRAIN_AUGS = Compose([
            RandomResizedCrop(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.5),
            Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)
VAL_AUGS =  Compose([
            Resize(CFG['img_size'], CFG['img_size']),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)
TEST_AUGS =  Compose([
            Resize(CFG['img_size'], CFG['img_size']),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)
# Worth trying: Blur, Noize,

In [8]:
from utils import seed_everything

seed_everything(2021)

In [9]:
data = pd.read_csv('/home/data/Cassava/train.csv')
print(data.shape)
# data.label.hist();

(21397, 2)


In [10]:
mapping = json.load(open('/home/data/Cassava/label_num_to_disease_map.json', 'r'))

In [11]:
data['fold'] = 0
#add random state
strkf = StratifiedKFold(n_splits=5, random_state=CFG['seed'])
_ =strkf.get_n_splits(data.image_id, data.label)
f = 0
for train_index, test_index in strkf.split(data.image_id, data.label):
    #print("TRAIN:", train_index, "TEST:", test_index)
    data.loc[data.index.isin(test_index), 'fold'] = f 
    f = f+1
data.fold.value_counts()

/usr/lib64/python3.6/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


1    4280
0    4280
4    4279
3    4279
2    4279
Name: fold, dtype: int64

In [15]:
from datasets import CassavaDataset
from train import Trainer
from model import CassvaImgClassifier

In [ ]:
device = torch.device('cuda')

for dev_fold in [1,2,3,4]:
    train_data = data[(data.fold != test_fold) & (data.fold != dev_fold)].reset_index(drop=True)
    val_data = data[data.fold == test_fold].reset_index(drop=True)
    dev_data = data[data.fold == dev_fold].reset_index(drop=True)  
    
    train_ds = CassavaDataset(train_data, TRAIN_AUGS, TEST_AUGS, mode = 'train')
    dev_ds = CassavaDataset(val_data, TRAIN_AUGS, TEST_AUGS, mode = 'val')
    valid_ds = CassavaDataset(val_data, TRAIN_AUGS, TEST_AUGS, mode = 'val')
    
    train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=CFG['train_bs'],
        pin_memory=False,
        drop_last=False,
        shuffle=True,        
        num_workers=8)
    
    val_loader = torch.utils.data.DataLoader(
        valid_ds, 
        batch_size=CFG['valid_bs'],
        num_workers=8,
        shuffle=False,
        pin_memory=False)
    
    dev_loader = torch.utils.data.DataLoader(
        dev_ds, 
        batch_size=CFG['valid_bs'],
        num_workers=8,
        shuffle=False,
        pin_memory=False)
    
    model = CassvaImgClassifier('tf_efficientnet_b4_ns', data.label.nunique(), pretrained=True).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4, weight_decay=1e-6)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=CFG['epochs'], T_mult=1, eta_min=1e-6, last_epoch=-1)
    trainer = Trainer(CFG, scheduler)
    
    loss_tr = nn.CrossEntropyLoss().to(device)
    loss_fn = nn.CrossEntropyLoss().to(device)
    for epoch in range(CFG['epochs']):
        print(f"epoch number {epoch}, lr = {optimizer.param_groups[0]['lr']}")
        trainer.train_one_epoch(model, optimizer, train_loader, loss_tr, epoch)
        val_acc, val_loss = trainer.valid_one_epoch(model, optimizer, val_loader, loss_fn, epoch)

        torch.save(model.state_dict(),'/home/samenko/Cassava/output/{}_dev_fold_{}_test_fold_{}_epoch_{}_val_loss_{:.4f}_val_acc_{:.4f}'.format(CFG['model_arch'], dev_fold, test_fold, epoch, val_acc, val_loss))

    #del model, optimizer, train_loader, val_loader, scaler, scheduler
    torch.cuda.empty_cache()

epoch number 0, lr = 0.0001


tain epoch 0 loss: 1.5850:   3%|▎         | 28/803 [00:17<06:47,  1.90it/s]